# Recurrent Neural Networks (RNN) - Complete Tutorial
## From Concepts to Implementation

Welcome to this comprehensive tutorial on Recurrent Neural Networks! This notebook covers:
1. **Core Concepts** - Understanding RNNs from the ground up
2. **Real-World Example** - Stock price prediction in StatLand
3. **Implementation from Scratch** - Building an RNN with NumPy
4. **Implementation with PyTorch** - Using modern libraries
5. **Visualizations** - Understanding the internal mechanisms
6. **The Vanishing/Exploding Gradient Problem**

---

## 1. Introduction: What Makes RNNs Special?

### The Problem RNNs Solve

Traditional neural networks have a **fixed number of inputs**:
- A network trained on images of 6×6 pixels can't handle 7×7 images
- A network with 3 inputs can't process 5 inputs

But many real-world problems involve **sequential data of varying lengths**:
- Stock prices over different time periods
- Sentences with different numbers of words
- Music sequences of varying durations

**Recurrent Neural Networks (RNNs)** solve this by using **feedback loops** that allow them to process sequences of any length!

### Key Features of RNNs

1. **Feedback Loops**: Output from one step becomes input to the next
2. **Shared Weights**: Same parameters used across all time steps
3. **Sequential Processing**: Process data one element at a time
4. **Memory**: Maintain information about previous inputs

---

## 2. The StatLand Stock Market Example

Let's use the example from the transcript: predicting stock prices in StatLand.

### StatLand Stock Market Rules:

1. **Low → Low**: If price is low for 2 days → stays low tomorrow
2. **Low → Medium**: If price goes from low to medium → goes high tomorrow
3. **High → Medium**: If price decreases from high to medium → goes lower tomorrow
4. **High → High**: If price stays high for 2 days → stays high tomorrow

### Data Encoding:
- **Low** = 0
- **Medium** = 0.5
- **High** = 1

---

In [ ]:
# Install required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
from matplotlib.patches import Rectangle, Circle, Arrow
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")

## 3. Visualizing Sequential Data

Let's visualize why we need RNNs - different sequences have different lengths!

In [ ]:
# Create sample stock price data with different lengths
np.random.seed(42)

# Blue company - 9 days of data
blue_company_days = np.arange(1, 10)
blue_company_prices = np.array([20, 22, 25, 28, 30, 32, 29, 27, 26])

# Red company - 5 days of data
red_company_days = np.arange(1, 6)
red_company_prices = np.array([15, 17, 19, 18, 20])

# Plotting
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(blue_company_days, blue_company_prices, 'o-', linewidth=3, 
        markersize=10, label='Blue Company (9 days)', color='#3498db')
ax.plot(red_company_days, red_company_prices, 's-', linewidth=3, 
        markersize=10, label='Red Company (5 days)', color='#e74c3c')

ax.set_xlabel('Day', fontsize=14, fontweight='bold')
ax.set_ylabel('Stock Price ($)', fontsize=14, fontweight='bold')
ax.set_title('Stock Prices Over Time - Different Sequence Lengths', 
             fontsize=16, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

# Add annotation
ax.annotate('Different lengths!\nWe need flexible networks', 
            xy=(7, 27), xytext=(5, 22),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'),
            fontsize=12, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

print(f"Blue Company has {len(blue_company_days)} data points")
print(f"Red Company has {len(red_company_days)} data points")
print("\n→ RNNs can handle BOTH sequences!")

## 4. RNN Architecture Visualization

### 4.1 Basic RNN with Feedback Loop

In [ ]:
def visualize_rnn_with_feedback():
    """Visualize RNN with feedback loop (folded representation)"""
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.axis('off')
    
    # Input node
    input_circle = Circle((2, 3), 0.5, color='lightblue', ec='black', lw=2)
    ax.add_patch(input_circle)
    ax.text(2, 3, 'Input\n(x)', ha='center', va='center', fontweight='bold')
    
    # Hidden node with feedback
    hidden_circle = Circle((5, 6), 0.7, color='lightcoral', ec='black', lw=2)
    ax.add_patch(hidden_circle)
    ax.text(5, 6, 'Hidden\nLayer', ha='center', va='center', fontweight='bold')
    
    # Output node
    output_circle = Circle((8, 3), 0.5, color='lightgreen', ec='black', lw=2)
    ax.add_patch(output_circle)
    ax.text(8, 3, 'Output\n(ŷ)', ha='center', va='center', fontweight='bold')
    
    # Arrows
    # Input to hidden
    ax.annotate('', xy=(4.5, 5.6), xytext=(2.4, 3.4),
                arrowprops=dict(arrowstyle='->', lw=2.5, color='blue'))
    ax.text(3, 4.8, 'W₁', fontsize=12, fontweight='bold', color='blue')
    
    # Hidden to output
    ax.annotate('', xy=(7.6, 3.4), xytext=(5.5, 5.6),
                arrowprops=dict(arrowstyle='->', lw=2.5, color='green'))
    ax.text(7, 4.8, 'W₃', fontsize=12, fontweight='bold', color='green')
    
    # Feedback loop - the key feature!
    from matplotlib.patches import FancyBboxPatch
    feedback_arrow = FancyArrowPatch((5.7, 6), (6.5, 7.5),
                                     connectionstyle="arc3,rad=.5",
                                     arrowstyle='->', lw=3, color='red')
    ax.add_patch(feedback_arrow)
    
    feedback_arrow2 = FancyArrowPatch((6.5, 7.5), (4.3, 6),
                                      connectionstyle="arc3,rad=-.5",
                                      arrowstyle='->', lw=3, color='red')
    ax.add_patch(feedback_arrow2)
    
    ax.text(6.5, 8, 'Feedback Loop\n(Memory!)', ha='center', 
            fontsize=12, fontweight='bold', color='red',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))
    ax.text(6.2, 7, 'W₂', fontsize=12, fontweight='bold', color='red')
    
    ax.set_title('RNN with Feedback Loop (Folded Representation)', 
                 fontsize=16, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.show()

visualize_rnn_with_feedback()
print("\n🔄 The feedback loop allows the network to remember previous inputs!")

### 4.2 Unrolled RNN - The Secret to Understanding!

**Unrolling** is the key to understanding RNNs. We create a copy of the network for each time step, which makes it easier to see how information flows through time.

In [ ]:
def visualize_unrolled_rnn(num_timesteps=3):
    """Visualize unrolled RNN across multiple timesteps"""
    fig, ax = plt.subplots(figsize=(16, 8))
    ax.set_xlim(0, num_timesteps * 4 + 1)
    ax.set_ylim(0, 10)
    ax.axis('off')
    
    colors = ['#ff9999', '#ffcc99', '#99ccff']
    time_labels = ['Yesterday\n(t-2)', 'Yesterday\n(t-1)', 'Today\n(t)']
    output_labels = ['Ignore', 'Ignore', 'Tomorrow\nPrediction!']
    
    for t in range(num_timesteps):
        x_offset = t * 4 + 2
        
        # Input
        input_circle = Circle((x_offset, 2), 0.4, color='lightblue', ec='black', lw=2)
        ax.add_patch(input_circle)
        ax.text(x_offset, 2, f'x_{t+1}', ha='center', va='center', fontweight='bold')
        ax.text(x_offset, 0.8, time_labels[t], ha='center', fontsize=9, style='italic')
        
        # Hidden layer
        hidden_circle = Circle((x_offset, 5), 0.5, color=colors[t], ec='black', lw=2)
        ax.add_patch(hidden_circle)
        ax.text(x_offset, 5, f'h_{t+1}', ha='center', va='center', fontweight='bold', fontsize=11)
        
        # Output
        output_circle = Circle((x_offset, 8), 0.4, color='lightgreen', ec='black', lw=2)
        ax.add_patch(output_circle)
        ax.text(x_offset, 8, f'ŷ_{t+1}', ha='center', va='center', fontweight='bold')
        ax.text(x_offset, 9.2, output_labels[t], ha='center', fontsize=9, 
                style='italic', fontweight='bold',
                color='red' if t < 2 else 'green')
        
        # Connections
        # Input to hidden
        ax.annotate('', xy=(x_offset, 4.5), xytext=(x_offset, 2.4),
                   arrowprops=dict(arrowstyle='->', lw=2, color='blue'))
        ax.text(x_offset + 0.3, 3.3, 'W₁', fontsize=10, color='blue', fontweight='bold')
        
        # Hidden to output
        ax.annotate('', xy=(x_offset, 7.6), xytext=(x_offset, 5.5),
                   arrowprops=dict(arrowstyle='->', lw=2, color='green'))
        ax.text(x_offset + 0.3, 6.5, 'W₃', fontsize=10, color='green', fontweight='bold')
        
        # Recurrent connection (except for last timestep)
        if t < num_timesteps - 1:
            ax.annotate('', xy=(x_offset + 4, 5), xytext=(x_offset + 0.5, 5),
                       arrowprops=dict(arrowstyle='->', lw=3, color='red'))
            ax.text(x_offset + 2, 5.5, 'W₂', fontsize=11, color='red', fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
    
    # Title and annotations
    ax.set_title('Unrolled RNN - Processing Sequential Data Through Time', 
                 fontsize=16, fontweight='bold', pad=20)
    
    # Add note about shared weights
    ax.text(num_timesteps * 2, 0.2, 
            '⚠️ Important: W₁, W₂, W₃, and biases are SHARED across all timesteps!',
            ha='center', fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.7', facecolor='lightyellow', 
                     edgecolor='orange', lw=2))
    
    plt.tight_layout()
    plt.show()

visualize_unrolled_rnn(3)
print("\n📊 Unrolling makes it easy to see how data flows through time!")
print("📌 Key insight: Same weights (W₁, W₂, W₃) used at every time step")

## 5. Mathematical Foundation

Let's break down the math behind RNNs step by step.

### RNN Equations

At each time step $t$:

1. **Hidden state update:**
   $$h_t = \tanh(W_1 \cdot x_t + W_2 \cdot h_{t-1} + b_1)$$

2. **Output:**
   $$\hat{y}_t = W_3 \cdot h_t + b_2$$

Where:
- $x_t$ = input at time $t$
- $h_t$ = hidden state at time $t$
- $h_{t-1}$ = hidden state from previous time step
- $W_1$ = input weight
- $W_2$ = recurrent weight (the feedback loop!)
- $W_3$ = output weight
- $b_1, b_2$ = biases

---

## 6. Implementation from Scratch

Let's build an RNN from scratch using only NumPy to truly understand how it works!

In [ ]:
class SimpleRNN:
    """
    Simple RNN implementation from scratch for stock price prediction.
    
    Architecture:
    - Input layer: 1 feature (stock price)
    - Hidden layer: configurable size with tanh activation
    - Output layer: 1 value (predicted next price)
    """
    
    def __init__(self, input_size=1, hidden_size=5, output_size=1, learning_rate=0.01):
        """
        Initialize RNN with random weights.
        
        Parameters:
        - input_size: Number of input features
        - hidden_size: Number of hidden units
        - output_size: Number of output values
        - learning_rate: Learning rate for gradient descent
        """
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        
        # Initialize weights with small random values
        # W1: input to hidden
        self.W1 = np.random.randn(hidden_size, input_size) * 0.01
        
        # W2: hidden to hidden (recurrent connection)
        self.W2 = np.random.randn(hidden_size, hidden_size) * 0.01
        
        # W3: hidden to output
        self.W3 = np.random.randn(output_size, hidden_size) * 0.01
        
        # Biases
        self.b1 = np.zeros((hidden_size, 1))
        self.b2 = np.zeros((output_size, 1))
        
        print("🧠 RNN initialized!")
        print(f"   Hidden size: {hidden_size}")
        print(f"   Total parameters: {self.count_parameters()}")
    
    def count_parameters(self):
        """Count total number of parameters."""
        return (self.W1.size + self.W2.size + self.W3.size + 
                self.b1.size + self.b2.size)
    
    def tanh(self, x):
        """Tanh activation function."""
        return np.tanh(x)
    
    def tanh_derivative(self, x):
        """Derivative of tanh."""
        return 1 - np.tanh(x) ** 2
    
    def forward(self, inputs):
        """
        Forward pass through the RNN.
        
        Parameters:
        - inputs: Sequence of inputs (list or array)
        
        Returns:
        - outputs: Predictions at each time step
        - hidden_states: Hidden states at each time step
        """
        T = len(inputs)  # Number of time steps
        
        # Store hidden states and outputs
        hidden_states = []
        outputs = []
        
        # Initialize hidden state
        h_prev = np.zeros((self.hidden_size, 1))
        
        # Process each time step
        for t in range(T):
            # Get input at time t
            x_t = np.array([[inputs[t]]])
            
            # Hidden state computation: h_t = tanh(W1*x_t + W2*h_{t-1} + b1)
            h_t = self.tanh(np.dot(self.W1, x_t) + 
                           np.dot(self.W2, h_prev) + 
                           self.b1)
            
            # Output computation: y_t = W3*h_t + b2
            y_t = np.dot(self.W3, h_t) + self.b2
            
            # Store for backpropagation
            hidden_states.append(h_t)
            outputs.append(y_t[0, 0])
            
            # Update h_prev for next iteration
            h_prev = h_t
        
        return outputs, hidden_states
    
    def backward(self, inputs, targets, outputs, hidden_states):
        """
        Backward pass - Backpropagation Through Time (BPTT).
        
        Parameters:
        - inputs: Input sequence
        - targets: Target values
        - outputs: Predicted outputs from forward pass
        - hidden_states: Hidden states from forward pass
        """
        T = len(inputs)
        
        # Initialize gradients
        dW1 = np.zeros_like(self.W1)
        dW2 = np.zeros_like(self.W2)
        dW3 = np.zeros_like(self.W3)
        db1 = np.zeros_like(self.b1)
        db2 = np.zeros_like(self.b2)
        
        # Initialize hidden state gradient
        dh_next = np.zeros((self.hidden_size, 1))
        
        # Backpropagate through time
        for t in reversed(range(T)):
            # Output error
            dy = outputs[t] - targets[t]
            
            # Gradient for W3 and b2
            dW3 += dy * hidden_states[t].T
            db2 += dy
            
            # Gradient flowing into hidden state
            dh = np.dot(self.W3.T, [[dy]]) + dh_next
            
            # Gradient through tanh
            dh_raw = dh * self.tanh_derivative(hidden_states[t])
            
            # Gradients for W1 and b1
            x_t = np.array([[inputs[t]]])
            dW1 += np.dot(dh_raw, x_t.T)
            db1 += dh_raw
            
            # Gradient for W2 (recurrent connection)
            if t > 0:
                dW2 += np.dot(dh_raw, hidden_states[t-1].T)
                dh_next = np.dot(self.W2.T, dh_raw)
            else:
                dh_next = np.zeros_like(dh_next)
        
        # Clip gradients to prevent exploding gradients
        for grad in [dW1, dW2, dW3, db1, db2]:
            np.clip(grad, -5, 5, out=grad)
        
        # Update weights
        self.W1 -= self.learning_rate * dW1
        self.W2 -= self.learning_rate * dW2
        self.W3 -= self.learning_rate * dW3
        self.b1 -= self.learning_rate * db1
        self.b2 -= self.learning_rate * db2
    
    def train(self, X_train, y_train, epochs=100, verbose=True):
        """
        Train the RNN on sequential data.
        
        Parameters:
        - X_train: List of input sequences
        - y_train: List of target sequences
        - epochs: Number of training epochs
        - verbose: Whether to print training progress
        """
        losses = []
        
        for epoch in range(epochs):
            total_loss = 0
            
            for inputs, targets in zip(X_train, y_train):
                # Forward pass
                outputs, hidden_states = self.forward(inputs)
                
                # Calculate loss (MSE)
                loss = np.mean([(o - t) ** 2 for o, t in zip(outputs, targets)])
                total_loss += loss
                
                # Backward pass
                self.backward(inputs, targets, outputs, hidden_states)
            
            avg_loss = total_loss / len(X_train)
            losses.append(avg_loss)
            
            if verbose and (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")
        
        return losses
    
    def predict(self, inputs):
        """Make predictions on a sequence."""
        outputs, _ = self.forward(inputs)
        return outputs[-1]  # Return only the final prediction

print("✓ SimpleRNN class defined!")

### 6.1 Prepare Training Data - StatLand Rules

In [ ]:
# Create training data based on StatLand rules
# Format: [yesterday, today] -> tomorrow

# Training sequences
X_train = [
    [0, 0],      # Low, Low
    [0, 0.5],    # Low, Medium
    [1, 0.5],    # High, Medium
    [1, 1],      # High, High
]

# Target outputs (tomorrow's price)
y_train = [
    [0, 0],      # -> Low
    [0.5, 1],    # -> High
    [0.5, 0],    # -> Low
    [1, 1],      # -> High
]

# Display training data
print("📚 StatLand Training Data:")
print("="*50)
rules = [
    "Low → Low ⇒ Low",
    "Low → Medium ⇒ High",
    "High → Medium ⇒ Low",
    "High → High ⇒ High"
]
for i, (x, y, rule) in enumerate(zip(X_train, y_train, rules)):
    print(f"{i+1}. {rule}")
    print(f"   Input:  {x} → Target: {y[-1]}")
print("="*50)

### 6.2 Train the RNN from Scratch

In [ ]:
# Create and train the RNN
print("🚀 Training RNN from Scratch...\n")

rnn = SimpleRNN(input_size=1, hidden_size=8, output_size=1, learning_rate=0.05)
losses = rnn.train(X_train, y_train, epochs=200, verbose=True)

print("\n✓ Training complete!")

### 6.3 Visualize Training Progress

In [ ]:
# Plot training loss
plt.figure(figsize=(12, 5))

plt.plot(losses, linewidth=2, color='#e74c3c')
plt.xlabel('Epoch', fontsize=12, fontweight='bold')
plt.ylabel('Loss (MSE)', fontsize=12, fontweight='bold')
plt.title('RNN Training Loss Over Time', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final Loss: {losses[-1]:.6f}")

### 6.4 Test the RNN

In [ ]:
# Test on training data
print("🧪 Testing RNN Predictions:")
print("="*60)

test_scenarios = [
    ([0, 0], 0, "Low → Low"),
    ([0, 0.5], 1, "Low → Medium"),
    ([1, 0.5], 0, "High → Medium"),
    ([1, 1], 1, "High → High")
]

for inputs, expected, scenario in test_scenarios:
    prediction = rnn.predict(inputs)
    expected_label = "Low" if expected == 0 else "High"
    pred_label = "Low" if prediction < 0.25 else ("High" if prediction > 0.75 else "Medium")
    
    print(f"\n{scenario} → Expected: {expected_label} ({expected})")
    print(f"   Predicted: {pred_label} ({prediction:.4f})")
    print(f"   ✓ Correct!" if abs(prediction - expected) < 0.3 else "   ✗ Needs more training")

print("="*60)

### 6.5 Visualize Predictions vs Actual

In [ ]:
# Create visualization of predictions
scenarios = ["Low→Low", "Low→Med", "High→Med", "High→High"]
expected_values = [0, 1, 0, 1]
predicted_values = [rnn.predict(x) for x in X_train]

x = np.arange(len(scenarios))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, expected_values, width, label='Expected', 
               color='#3498db', alpha=0.8, edgecolor='black', linewidth=2)
bars2 = ax.bar(x + width/2, predicted_values, width, label='Predicted', 
               color='#e74c3c', alpha=0.8, edgecolor='black', linewidth=2)

ax.set_xlabel('Scenario', fontsize=12, fontweight='bold')
ax.set_ylabel('Stock Price (Normalized)', fontsize=12, fontweight='bold')
ax.set_title('RNN Predictions vs Expected Values', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(scenarios)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}',
                ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Implementation with PyTorch

Now let's implement the same RNN using PyTorch, a modern deep learning library!

In [ ]:
# Install and import PyTorch
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    print("✓ PyTorch imported successfully!")
except ImportError:
    print("Installing PyTorch...")
    import sys
    !{sys.executable} -m pip install torch --break-system-packages
    import torch
    import torch.nn as nn
    import torch.optim as optim
    print("✓ PyTorch installed and imported!")

print(f"PyTorch version: {torch.__version__}")

In [ ]:
class PyTorchRNN(nn.Module):
    """
    RNN implementation using PyTorch.
    """
    
    def __init__(self, input_size=1, hidden_size=8, output_size=1):
        super(PyTorchRNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        # RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        
        # Output layer
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden=None):
        """
        Forward pass.
        
        Parameters:
        - x: Input tensor of shape (batch_size, sequence_length, input_size)
        - hidden: Initial hidden state
        
        Returns:
        - output: Predictions
        - hidden: Final hidden state
        """
        # RNN forward pass
        rnn_out, hidden = self.rnn(x, hidden)
        
        # Get output from the last time step
        last_output = rnn_out[:, -1, :]
        
        # Pass through output layer
        output = self.fc(last_output)
        
        return output, hidden

print("✓ PyTorchRNN class defined!")

### 7.1 Prepare Data for PyTorch

In [ ]:
# Convert data to PyTorch tensors
X_train_torch = [torch.FloatTensor([[x] for x in seq]).unsqueeze(0) for seq in X_train]
y_train_torch = [torch.FloatTensor([[seq[-1]]]) for seq in y_train]

print("Sample input shape:", X_train_torch[0].shape)
print("Sample target shape:", y_train_torch[0].shape)
print("\n✓ Data prepared for PyTorch!")

### 7.2 Train PyTorch RNN

In [ ]:
# Initialize model
model = PyTorchRNN(input_size=1, hidden_size=8, output_size=1)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training
print("🚀 Training PyTorch RNN...\n")
epochs = 200
losses_pytorch = []

for epoch in range(epochs):
    total_loss = 0
    
    for x, y in zip(X_train_torch, y_train_torch):
        # Forward pass
        optimizer.zero_grad()
        output, _ = model(x)
        
        # Calculate loss
        loss = criterion(output, y)
        total_loss += loss.item()
        
        # Backward pass
        loss.backward()
        optimizer.step()
    
    avg_loss = total_loss / len(X_train_torch)
    losses_pytorch.append(avg_loss)
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")

print("\n✓ Training complete!")

### 7.3 Compare Training Performance

In [ ]:
# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# From scratch
ax1.plot(losses, linewidth=2, color='#e74c3c')
ax1.set_xlabel('Epoch', fontweight='bold')
ax1.set_ylabel('Loss', fontweight='bold')
ax1.set_title('From Scratch (NumPy)', fontweight='bold', fontsize=13)
ax1.grid(True, alpha=0.3)

# PyTorch
ax2.plot(losses_pytorch, linewidth=2, color='#3498db')
ax2.set_xlabel('Epoch', fontweight='bold')
ax2.set_ylabel('Loss', fontweight='bold')
ax2.set_title('PyTorch Implementation', fontweight='bold', fontsize=13)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"From Scratch - Final Loss: {losses[-1]:.6f}")
print(f"PyTorch - Final Loss: {losses_pytorch[-1]:.6f}")

### 7.4 Test PyTorch Model

In [ ]:
# Test PyTorch model
model.eval()

print("🧪 Testing PyTorch RNN Predictions:")
print("="*60)

with torch.no_grad():
    for i, (x, y, scenario) in enumerate(zip(X_train_torch, y_train_torch, 
                                              ["Low→Low", "Low→Med", "High→Med", "High→High"])):
        prediction, _ = model(x)
        pred_val = prediction.item()
        expected_val = y.item()
        
        expected_label = "Low" if expected_val == 0 else "High"
        pred_label = "Low" if pred_val < 0.25 else ("High" if pred_val > 0.75 else "Medium")
        
        print(f"\n{scenario} → Expected: {expected_label} ({expected_val})")
        print(f"   Predicted: {pred_label} ({pred_val:.4f})")
        print(f"   ✓ Correct!" if abs(pred_val - expected_val) < 0.3 else "   ✗ Needs more training")

print("="*60)

## 8. The Vanishing/Exploding Gradient Problem

This is the **biggest challenge** with vanilla RNNs and explains why they're rarely used in practice!

### The Problem

When we unroll an RNN many times (long sequences), the gradient gets multiplied by the recurrent weight $W_2$ repeatedly:

$$\text{Gradient} \propto W_2^T$$

Where $T$ is the number of time steps.

- If $|W_2| > 1$ → **Exploding Gradients** 💥
- If $|W_2| < 1$ → **Vanishing Gradients** 👻

---

In [ ]:
def demonstrate_gradient_problem():
    """
    Visualize how gradients explode or vanish based on W2 value.
    """
    timesteps = np.arange(1, 51)
    
    # Different values of W2
    w2_values = [0.5, 0.9, 1.0, 1.1, 2.0]
    colors = ['#3498db', '#9b59b6', '#95a5a6', '#e67e22', '#e74c3c']
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Linear scale
    for w2, color in zip(w2_values, colors):
        gradient_magnitude = w2 ** timesteps
        ax1.plot(timesteps, gradient_magnitude, linewidth=2.5, 
                label=f'W₂ = {w2}', color=color)
    
    ax1.set_xlabel('Number of Time Steps', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Gradient Magnitude', fontsize=12, fontweight='bold')
    ax1.set_title('Gradient Magnitude (Linear Scale)', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([0, 20])
    
    # Add annotations
    ax1.axhline(y=1, color='green', linestyle='--', linewidth=2, label='Ideal')
    ax1.text(25, 15, 'EXPLODING\nGRADIENTS\n💥', fontsize=12, 
            fontweight='bold', color='red',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))
    
    # Log scale
    for w2, color in zip(w2_values, colors):
        gradient_magnitude = w2 ** timesteps
        ax2.semilogy(timesteps, gradient_magnitude, linewidth=2.5, 
                    label=f'W₂ = {w2}', color=color)
    
    ax2.set_xlabel('Number of Time Steps', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Gradient Magnitude (log scale)', fontsize=12, fontweight='bold')
    ax2.set_title('Gradient Magnitude (Log Scale)', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    # Add annotations
    ax2.axhline(y=1, color='green', linestyle='--', linewidth=2)
    ax2.text(25, 1e-10, 'VANISHING\nGRADIENTS\n👻', fontsize=12, 
            fontweight='bold', color='blue',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.7))
    
    plt.tight_layout()
    plt.show()

demonstrate_gradient_problem()

print("\n⚠️ Key Insights:")
print("   • W₂ > 1: Gradients EXPLODE exponentially (red/orange lines)")
print("   • W₂ < 1: Gradients VANISH exponentially (blue/purple lines)")
print("   • W₂ = 1: Stable, but loses expressiveness")
print("\n💡 Solution: Long Short-Term Memory (LSTM) networks!")

### 8.1 Numerical Demonstration

In [ ]:
# Show exact numbers
print("🔢 Numerical Examples:\n")
print("="*60)

timesteps_examples = [10, 20, 50]
w2_examples = [0.5, 2.0]

for w2 in w2_examples:
    problem_type = "VANISHING" if w2 < 1 else "EXPLODING"
    print(f"\nW₂ = {w2} ({problem_type} Gradients):")
    print("-" * 40)
    
    for t in timesteps_examples:
        magnitude = w2 ** t
        print(f"  After {t:2d} timesteps: {magnitude:.2e}")
    
    if w2 < 1:
        print(f"  → Gradient becomes ≈0, can't learn long-term!")
    else:
        print(f"  → Gradient explodes, can't converge!")

print("\n" + "="*60)

### 8.2 Visualize the Impact on Optimization

In [ ]:
def visualize_optimization_landscape():
    """
    Show how exploding/vanishing gradients affect optimization.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Create a simple loss landscape
    x = np.linspace(-2, 2, 100)
    loss = x**2  # Simple quadratic loss
    
    # Exploding gradients - large steps
    ax1.plot(x, loss, 'b-', linewidth=3, label='Loss Function')
    
    # Simulate gradient descent with large steps
    positions_explode = [-1.5, 1.8, -1.9, 1.7, -1.6, 2.0]  # Bouncing around
    for i, pos in enumerate(positions_explode):
        ax1.plot(pos, pos**2, 'ro', markersize=12)
        if i < len(positions_explode) - 1:
            ax1.annotate('', xy=(positions_explode[i+1], positions_explode[i+1]**2),
                        xytext=(pos, pos**2),
                        arrowprops=dict(arrowstyle='->', lw=2, color='red'))
    
    ax1.plot(0, 0, 'g*', markersize=20, label='Optimal')
    ax1.set_xlabel('Parameter Value', fontweight='bold', fontsize=12)
    ax1.set_ylabel('Loss', fontweight='bold', fontsize=12)
    ax1.set_title('EXPLODING Gradients\n(Steps Too Large)', 
                  fontweight='bold', fontsize=13, color='red')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    ax1.text(0, 3.5, '💥 Bouncing around!\nCannot converge', 
            ha='center', fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))
    
    # Vanishing gradients - tiny steps
    ax2.plot(x, loss, 'b-', linewidth=3, label='Loss Function')
    
    # Simulate gradient descent with tiny steps
    positions_vanish = [-1.5, -1.48, -1.46, -1.44, -1.42, -1.40]  # Very slow
    for i, pos in enumerate(positions_vanish):
        ax2.plot(pos, pos**2, 'ro', markersize=12)
        if i < len(positions_vanish) - 1:
            ax2.annotate('', xy=(positions_vanish[i+1], positions_vanish[i+1]**2),
                        xytext=(pos, pos**2),
                        arrowprops=dict(arrowstyle='->', lw=2, color='red'))
    
    ax2.plot(0, 0, 'g*', markersize=20, label='Optimal')
    ax2.set_xlabel('Parameter Value', fontweight='bold', fontsize=12)
    ax2.set_ylabel('Loss', fontweight='bold', fontsize=12)
    ax2.set_title('VANISHING Gradients\n(Steps Too Small)', 
                  fontweight='bold', fontsize=13, color='blue')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    ax2.text(0, 3.5, '👻 Moving too slowly!\nNever reaches optimum', 
            ha='center', fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.7))
    
    plt.tight_layout()
    plt.show()

visualize_optimization_landscape()

print("\n⚠️ Both problems prevent effective training!")
print("   • Exploding: Takes huge steps, bounces around")
print("   • Vanishing: Takes tiny steps, gets stuck")

## 9. Solutions to the Gradient Problem

### 9.1 Gradient Clipping (Partial Solution)

A simple technique to prevent exploding gradients:

In [ ]:
def demonstrate_gradient_clipping():
    """
    Show how gradient clipping helps with exploding gradients.
    """
    # Simulate gradients
    np.random.seed(42)
    original_gradients = np.random.randn(100) * 10  # Some very large gradients
    
    # Apply clipping
    clip_value = 5.0
    clipped_gradients = np.clip(original_gradients, -clip_value, clip_value)
    
    # Visualize
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Original gradients
    ax1.hist(original_gradients, bins=30, color='#e74c3c', alpha=0.7, edgecolor='black')
    ax1.axvline(-clip_value, color='orange', linestyle='--', linewidth=3, label='Clip threshold')
    ax1.axvline(clip_value, color='orange', linestyle='--', linewidth=3)
    ax1.set_xlabel('Gradient Value', fontweight='bold')
    ax1.set_ylabel('Frequency', fontweight='bold')
    ax1.set_title('Original Gradients (Some Exploding!)', fontweight='bold', fontsize=13)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Clipped gradients
    ax2.hist(clipped_gradients, bins=30, color='#3498db', alpha=0.7, edgecolor='black')
    ax2.axvline(-clip_value, color='green', linestyle='--', linewidth=3, label='Clip threshold')
    ax2.axvline(clip_value, color='green', linestyle='--', linewidth=3)
    ax2.set_xlabel('Gradient Value', fontweight='bold')
    ax2.set_ylabel('Frequency', fontweight='bold')
    ax2.set_title('Clipped Gradients (Controlled!)', fontweight='bold', fontsize=13)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Original gradient range: [{original_gradients.min():.2f}, {original_gradients.max():.2f}]")
    print(f"Clipped gradient range:  [{clipped_gradients.min():.2f}, {clipped_gradients.max():.2f}]")
    print(f"\n✓ Clipping prevents exploding gradients!")
    print(f"⚠️ But doesn't solve vanishing gradients...")

demonstrate_gradient_clipping()

### 9.2 Better Initialization

In [ ]:
# Compare different weight initializations
def compare_initializations():
    timesteps = np.arange(1, 31)
    
    # Different initialization strategies
    init_strategies = {
        'Random [-1, 1]': np.random.uniform(-1, 1),
        'Xavier': np.sqrt(1.0 / 8),  # sqrt(1/hidden_size)
        'He': np.sqrt(2.0 / 8),
    }
    
    plt.figure(figsize=(12, 6))
    
    for name, w2 in init_strategies.items():
        magnitude = np.abs(w2) ** timesteps
        plt.semilogy(timesteps, magnitude, linewidth=2.5, label=f'{name} (W₂={w2:.3f})')
    
    plt.axhline(y=1, color='green', linestyle='--', linewidth=2, label='Ideal (=1)')
    plt.xlabel('Number of Time Steps', fontsize=12, fontweight='bold')
    plt.ylabel('Gradient Magnitude (log scale)', fontsize=12, fontweight='bold')
    plt.title('Effect of Weight Initialization on Gradient Flow', 
              fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("✓ Better initialization helps, but doesn't fully solve the problem")

compare_initializations()

### 9.3 The Real Solution: LSTM & GRU

The **real solution** to vanishing/exploding gradients is to use:

1. **LSTM (Long Short-Term Memory)** - Introduced in 1997
2. **GRU (Gated Recurrent Unit)** - Introduced in 2014

These architectures use **gates** to control information flow and prevent gradients from vanishing!

---

## 10. Real-World Applications

### Where are RNNs (and their variants) used?

1. **Natural Language Processing**
   - Machine translation
   - Text generation
   - Sentiment analysis

2. **Time Series Prediction**
   - Stock prices
   - Weather forecasting
   - Sales forecasting

3. **Speech Recognition**
   - Converting speech to text
   - Voice assistants

4. **Video Analysis**
   - Action recognition
   - Video captioning

5. **Music Generation**
   - Composing melodies
   - Style transfer

---

## 11. Extended Example: Multi-Day Stock Prediction

In [ ]:
# Generate a longer sequence
np.random.seed(42)

# Create a more complex stock pattern
def generate_stock_sequence(length=10):
    """
    Generate a stock price sequence following StatLand rules with some noise.
    """
    prices = [np.random.choice([0, 0.5, 1])]
    
    for i in range(1, length):
        prev_price = prices[-1]
        
        if i >= 2:
            prev_prev_price = prices[-2]
        else:
            prev_prev_price = prev_price
        
        # Apply StatLand rules with some randomness
        if prev_prev_price == 0 and prev_price == 0:
            next_price = 0 if np.random.rand() < 0.8 else 0.5
        elif prev_prev_price == 0 and prev_price == 0.5:
            next_price = 1 if np.random.rand() < 0.8 else 0.5
        elif prev_prev_price == 1 and prev_price == 0.5:
            next_price = 0 if np.random.rand() < 0.8 else 0.5
        elif prev_prev_price == 1 and prev_price == 1:
            next_price = 1 if np.random.rand() < 0.8 else 0.5
        else:
            next_price = np.random.choice([0, 0.5, 1])
        
        prices.append(next_price)
    
    return prices

# Generate sequences
sequence_length = 15
stock_sequence = generate_stock_sequence(sequence_length)

# Visualize
plt.figure(figsize=(14, 6))
days = np.arange(1, len(stock_sequence) + 1)

plt.plot(days, stock_sequence, 'o-', linewidth=3, markersize=10, color='#3498db')
plt.xlabel('Day', fontsize=12, fontweight='bold')
plt.ylabel('Stock Price (Normalized)', fontsize=12, fontweight='bold')
plt.title('Extended Stock Price Sequence in StatLand', fontsize=14, fontweight='bold')
plt.yticks([0, 0.5, 1], ['Low (0)', 'Medium (0.5)', 'High (1)'])
plt.grid(True, alpha=0.3)

# Add annotations for patterns
for i in range(1, len(stock_sequence)):
    if i >= 2:
        if stock_sequence[i-2] == 0 and stock_sequence[i-1] == 0 and stock_sequence[i] == 0:
            plt.annotate('L→L→L', xy=(i, stock_sequence[i]), 
                        xytext=(i, stock_sequence[i]-0.15),
                        fontsize=8, ha='center', color='blue')

plt.tight_layout()
plt.show()

print(f"Generated sequence of length {len(stock_sequence)}:")
print(stock_sequence)

### Test RNN on Longer Sequence

In [ ]:
# Use our trained RNN to make predictions on this longer sequence
def sliding_window_predictions(sequence, window_size=2):
    """
    Make predictions using a sliding window approach.
    """
    predictions = []
    
    for i in range(len(sequence) - window_size):
        window = sequence[i:i+window_size]
        pred = rnn.predict(window)
        predictions.append(pred)
    
    return predictions

# Make predictions
window_size = 2
predictions = sliding_window_predictions(stock_sequence, window_size)
actual = stock_sequence[window_size:]

# Visualize predictions vs actual
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Predictions vs Actual
days = np.arange(window_size + 1, len(stock_sequence) + 1)
ax1.plot(days, actual, 'o-', linewidth=3, markersize=10, 
         label='Actual', color='#2ecc71', alpha=0.8)
ax1.plot(days, predictions, 's--', linewidth=2, markersize=8, 
         label='Predicted', color='#e74c3c', alpha=0.8)
ax1.set_xlabel('Day', fontsize=12, fontweight='bold')
ax1.set_ylabel('Stock Price', fontsize=12, fontweight='bold')
ax1.set_title('RNN Predictions on Extended Sequence', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Plot 2: Prediction Error
errors = [abs(p - a) for p, a in zip(predictions, actual)]
ax2.bar(days, errors, color='#e74c3c', alpha=0.6, edgecolor='black')
ax2.axhline(y=0.25, color='orange', linestyle='--', linewidth=2, 
            label='Acceptable error threshold')
ax2.set_xlabel('Day', fontsize=12, fontweight='bold')
ax2.set_ylabel('Absolute Error', fontsize=12, fontweight='bold')
ax2.set_title('Prediction Errors', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Calculate metrics
mae = np.mean(errors)
mse = np.mean([e**2 for e in errors])
accurate_predictions = sum([1 for e in errors if e < 0.25])
accuracy = accurate_predictions / len(errors) * 100

print(f"\n📊 Performance Metrics:")
print(f"   Mean Absolute Error: {mae:.4f}")
print(f"   Mean Squared Error:  {mse:.4f}")
print(f"   Accuracy (< 0.25 error): {accuracy:.1f}%")
print(f"   Correct predictions: {accurate_predictions}/{len(errors)}")

## 12. Summary and Key Takeaways

### ✅ What We Learned

1. **RNNs are designed for sequential data**
   - Handle variable-length inputs
   - Use feedback loops for memory
   - Share weights across time steps

2. **Unrolling reveals the structure**
   - Makes it easier to understand information flow
   - Each time step is a copy of the same network
   - Shared parameters across all copies

3. **The Vanishing/Exploding Gradient Problem**
   - Major limitation of vanilla RNNs
   - Prevents learning long-term dependencies
   - Solved by LSTM and GRU architectures

4. **Implementation Approaches**
   - From scratch: Understand the fundamentals
   - With libraries: Practical applications

### 🎯 Next Steps

1. **Learn about LSTM networks** - The modern solution to gradient problems
2. **Explore GRU networks** - Simpler alternative to LSTMs
3. **Study Transformers** - State-of-the-art for many sequence tasks
4. **Apply to real datasets** - Try time series, NLP, or other sequential data

### 📚 Additional Resources

- StatQuest Videos: Excellent visual explanations
- "Understanding LSTM Networks" by Christopher Olah
- PyTorch/TensorFlow documentation for practical implementations

---

## 🎉 Congratulations!

You've completed a comprehensive tutorial on Recurrent Neural Networks! You now understand:
- How RNNs work internally
- Why they're useful for sequential data
- Their limitations and solutions
- How to implement them from scratch and with libraries

**Keep learning and Quest on! 🚀**

---

## Bonus: Interactive RNN Playground

Try experimenting with different parameters!

In [ ]:
def rnn_playground(hidden_size=8, learning_rate=0.05, epochs=200):
    """
    Interactive function to experiment with RNN parameters.
    
    Try different values:
    - hidden_size: 2, 4, 8, 16, 32
    - learning_rate: 0.001, 0.01, 0.05, 0.1
    - epochs: 50, 100, 200, 500
    """
    print(f"\n🎮 RNN Playground")
    print(f"   Hidden size: {hidden_size}")
    print(f"   Learning rate: {learning_rate}")
    print(f"   Epochs: {epochs}\n")
    
    # Train RNN with given parameters
    test_rnn = SimpleRNN(input_size=1, hidden_size=hidden_size, 
                        output_size=1, learning_rate=learning_rate)
    losses = test_rnn.train(X_train, y_train, epochs=epochs, verbose=False)
    
    # Visualize results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Training curve
    ax1.plot(losses, linewidth=2, color='#3498db')
    ax1.set_xlabel('Epoch', fontweight='bold')
    ax1.set_ylabel('Loss', fontweight='bold')
    ax1.set_title(f'Training Loss (Final: {losses[-1]:.6f})', fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Predictions
    predictions = [test_rnn.predict(x) for x in X_train]
    expected = [y[-1] for y in y_train]
    
    x_pos = np.arange(len(X_train))
    width = 0.35
    
    ax2.bar(x_pos - width/2, expected, width, label='Expected', 
           color='#2ecc71', alpha=0.8)
    ax2.bar(x_pos + width/2, predictions, width, label='Predicted', 
           color='#e74c3c', alpha=0.8)
    
    ax2.set_xlabel('Scenario', fontweight='bold')
    ax2.set_ylabel('Value', fontweight='bold')
    ax2.set_title('Predictions vs Expected', fontweight='bold')
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(['L→L', 'L→M', 'H→M', 'H→H'])
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate accuracy
    errors = [abs(p - e) for p, e in zip(predictions, expected)]
    accuracy = sum([1 for e in errors if e < 0.25]) / len(errors) * 100
    
    print(f"\n✓ Accuracy: {accuracy:.1f}%")
    print(f"✓ Avg Error: {np.mean(errors):.4f}")
    
    return test_rnn, losses

# Try it out!
print("\n" + "="*60)
print("Try experimenting with different parameters:")
print("  rnn_playground(hidden_size=16, learning_rate=0.01, epochs=300)")
print("="*60)

# Example run
rnn_playground(hidden_size=8, learning_rate=0.05, epochs=200)

---

# 🎓 End of Tutorial

**Thank you for learning about Recurrent Neural Networks!**

Remember:
- RNNs are great for sequential data
- They use feedback loops to create memory
- Vanilla RNNs have gradient problems
- LSTM and GRU solve these problems

**Keep learning, keep building, and Quest on! 🚀**

---